In [2]:
import pandas as pd
import warnings
import re
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_parquet("archivos_parquet_florida/1_METADATA_FLORIDA_SIN_FILTRAR.parquet")
#tardo 47 segundos

In [4]:
df.shape

(3025011, 15)

In [5]:
import re

# Convertir la columna 'category' a tipo string
df['category'] = df['category'].astype(str)

# Crear una máscara booleana donde cada valor es True si el string "restaurant" está presente
mask = df['category'].apply(lambda x: re.search(r'restaurant', x, re.IGNORECASE) is not None)

# Filtrar el DataFrame usando la máscara
filtered_df = df[mask]

#tardo 1minuto 5 segundos

In [6]:
filtered_df.shape

(212014, 15)

In [7]:
import geopandas as gpd
from shapely.geometry import Point

# Cargar el shapefile o GeoJSON de Florida
usa = gpd.read_file("st12_d00_shp/st12_d00.shp")
florida = usa[usa['NAME'] == 'Florida']

# Convierte tu DataFrame a un GeoDataFrame
geometry = [Point(xy) for xy in zip(filtered_df['longitude'], filtered_df['latitude'])]
geo_df = gpd.GeoDataFrame(filtered_df, geometry=geometry)

# Filtra solo aquellos puntos que están dentro de Florida
florida_points = geo_df[geo_df.geometry.within(florida.geometry.iloc[0])]

florida_points.shape

(13748, 16)

In [8]:
florida_points.shape

(13748, 16)

In [9]:
florida_points.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 13748 entries, 573 to 3024720
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   name              13748 non-null  object  
 1   address           13661 non-null  object  
 2   gmap_id           13748 non-null  object  
 3   description       4636 non-null   object  
 4   latitude          13748 non-null  float64 
 5   longitude         13748 non-null  float64 
 6   category          13748 non-null  object  
 7   avg_rating        13748 non-null  float64 
 8   num_of_reviews    13748 non-null  int64   
 9   price             5972 non-null   object  
 10  hours             12152 non-null  object  
 11  MISC              13668 non-null  object  
 12  state             12193 non-null  object  
 13  relative_results  10450 non-null  object  
 14  url               13748 non-null  object  
 15  geometry          13748 non-null  geometry
dtypes: float64

In [10]:
import os
import pandas as pd

path = 'review-Florida'
json_files = [file for file in os.listdir(path) if file.endswith('.json')]

dfs = []

for file in json_files:
    try:
        df = pd.read_json(os.path.join(path, file),lines=True)
        dfs.append(df)
    except ValueError as e:
        print(f"Error al leer el archivo {file}: {e}")

# Si todos los archivos se leen correctamente, se concatena
if len(dfs) == len(json_files):
    df_reviews = pd.concat(dfs, ignore_index=True)
    df_reviews.head()
else:
    print("Algunos archivos tienen problemas y no se pueden leer.")


#tardo 56 segundos

In [11]:
df_reviews.shape

(2850000, 8)

In [12]:
result = florida_points.merge(df_reviews, on='gmap_id', how='left')
# tardo 8 segundos

In [13]:
result.shape

(1007300, 23)

In [14]:
result.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 1007300 entries, 0 to 1007299
Data columns (total 23 columns):
 #   Column            Non-Null Count    Dtype   
---  ------            --------------    -----   
 0   name_x            1007300 non-null  object  
 1   address           1006869 non-null  object  
 2   gmap_id           1007300 non-null  object  
 3   description       851181 non-null   object  
 4   latitude          1007300 non-null  float64 
 5   longitude         1007300 non-null  float64 
 6   category          1007300 non-null  object  
 7   avg_rating        1007300 non-null  float64 
 8   num_of_reviews    1007300 non-null  int64   
 9   price             920640 non-null   object  
 10  hours             991539 non-null   object  
 11  MISC              1006778 non-null  object  
 12  state             991877 non-null   object  
 13  relative_results  980279 non-null   object  
 14  url               1007300 non-null  object  
 15  geometry          100730

In [15]:
result.shape

(1007300, 23)

In [16]:
filtered_result = result[result['text'].notna() & (result['text'] != "")]

In [17]:
filtered_df.shape

(212014, 15)

In [18]:
num_unique_addresses = filtered_result['address'].nunique()
print(num_unique_addresses)


5136


In [19]:
num_unique_text = filtered_result['text'].nunique()
print(num_unique_text)


483588


In [20]:
# Encuentra los valores duplicados en la columna 'text'
duplicate_texts = filtered_result[filtered_result['text'].duplicated(keep=False)]['text']

# Obtén los 4 valores más repetidos
top_duplicated_texts = duplicate_texts.value_counts().head(4).index

# Filtra el DataFrame original para mostrar solo esos valores repetidos
examples = filtered_result[filtered_result['text'].isin(top_duplicated_texts)]

examples.text


8          Great food
12          Good food
13               Good
193              Good
199        Great food
              ...    
1005186    Great food
1005364     Good food
1005796     Good food
1005842    Great food
1005857    Great food
Name: text, Length: 9134, dtype: object

In [21]:
has_empty_values = filtered_result['text'].isna().any() or (filtered_result['text'] == "").any()

if has_empty_values:
    print("La columna 'text' tiene al menos un valor que es None, NaN o una cadena vacía.")
else:
    print("La columna 'text' no tiene valores None, NaN o cadenas vacías.")


La columna 'text' no tiene valores None, NaN o cadenas vacías.


In [22]:
# Suponiendo que tu DataFrame se llama 'joined'
unique_addresses = filtered_result.drop_duplicates(subset='address', keep='first')


In [23]:
import folium

# Crea un mapa centrado alrededor de Florida
florida_map = folium.Map(location=[27.9944024, -81.7602544], zoom_start=7) 

# Agrega cada punto del dataframe al mapa
for _, row in unique_addresses.iterrows():
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=3,
        fill=True,
        fill_color="blue",
        fill_opacity=0.6
    ).add_to(florida_map)

florida_map
#tardo 5 minutos 24 segundos


In [24]:
filtered_result.shape

(616166, 23)

In [ ]:
print(filtered_result['time'].head())

In [26]:
import pandas as pd

# Supongamos que tienes un DataFrame llamado df y la columna con timestamps UNIX en milisegundos se llama 'unix_timestamp'
filtered_result['time'] = pd.to_datetime(filtered_result['time'], unit='ms').dt.strftime('%d/%m/%Y')


In [27]:
filtered_result.head(1)

,name_x,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,...,relative_results,url,geometry,user_id,name_y,time,rating,text,pics,resp
1,Honduras Nica Cafe,"Honduras Nica Cafe, 280 NW 79th St, Miami, FL ...",0x88d9b176ea025e13:0x35d50186b9d6b2cc,None,25.847143,-80.20237,['Honduran restaurant'],3.5,84,None,...,"[0x88d9b0d4f164e5b1:0xdcc57d188304c503, 0x88d9...",https://www.google.com/maps/place//data=!4m2!3...,POINT (-80.20237 25.84714),1.057252e+20,Cindys 4luvz,17/04/2021,4.0,The nacatamal is a good size and taste ok. The...,None,None


In [30]:
# Lista de columnas a eliminar
columns_to_drop = ['gmap_id', 'pics', 'resp','user_id','geometry','url','relative_results']  # Reemplaza los nombres con los nombres reales de tus columnas

# Eliminar las columnas del DataFrame
filtered_result = filtered_result.drop(columns=columns_to_drop)


In [32]:
filtered_result = filtered_result.rename(columns={
    'name_x': 'business_name',
    'name_y': 'user_name'
})


In [33]:
filtered_result.head(1)

,business_name,address,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,user_name,time,rating,text
1,Honduras Nica Cafe,"Honduras Nica Cafe, 280 NW 79th St, Miami, FL ...",None,25.847143,-80.20237,['Honduran restaurant'],3.5,84,None,"[[Thursday, 7AM–9PM], [Friday, 7AM–10PM], [Sat...",{'Accessibility': ['Wheelchair accessible entr...,Closes soon ⋅ 9PM ⋅ Opens 7AM Fri,Cindys 4luvz,17/04/2021,4.0,The nacatamal is a good size and taste ok. The...


In [34]:
filtered_result.shape

(616166, 16)

In [35]:
filtered_result.to_parquet("archivos_parquet_florida/2_ETL_FLORIDA_FILTRADO_x_RESTO_x_MAPA_x_TEXTO.parquet")
